In [1]:
import numpy as np, pandas as pd

In [2]:
data = pd.read_excel('./Bike Sales Dashboard.xlsx',sheet_name='bike_buyers')

In [3]:
data.isna().sum()

ID                  0
Martial Status      0
Gender              0
Income              0
Children            0
Education           0
Occupation          0
Home Owner          0
Cars                0
Commute Distance    0
Region              0
Age                 0
Age Brackets        0
Purchased Bike      0
dtype: int64

In [4]:
data.head()

,ID,Martial Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Age Brackets,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,Middle Age,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,Middle Age,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,Old,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Middle Age,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Middle Age,Yes


In [5]:
data = pd.read_excel('./Bike Sales Dashboard.xlsx',sheet_name='bike_buyers')
data['Martial Status'] = data['Martial Status'].map({'Married': 1, 'Single': 0})
data['Home Owner'] = data['Home Owner'].map({'Yes': 1, 'No': 0})
data['Purchased Bike'] = data['Purchased Bike'].map({'Yes': 1, 'No': 0})
data['Gender'] = data['Gender'].map({'Male': 1, 'Female':0})

In [6]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
data.head()

,ID,Martial Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Age Brackets,Purchased Bike
0,12496,1,0,40000,1,Bachelors,Skilled Manual,1,0,0-1 Miles,Europe,42,Middle Age,0
1,24107,1,1,30000,3,Partial College,Clerical,1,1,0-1 Miles,Europe,43,Middle Age,0
2,14177,1,1,80000,5,Partial College,Professional,0,2,2-5 Miles,Europe,60,Old,0
3,24381,0,1,70000,0,Bachelors,Professional,1,1,5-10 Miles,Pacific,41,Middle Age,1
4,25597,0,1,30000,0,Bachelors,Clerical,0,0,0-1 Miles,Europe,36,Middle Age,1


In [8]:
ohe_education = OneHotEncoder().fit_transform(data['Education'].values.reshape(-1, 1)).toarray()
ohe_occ = OneHotEncoder().fit_transform(data['Occupation'].values.reshape(-1, 1)).toarray()
ohe_commute = OneHotEncoder().fit_transform(data['Commute Distance'].values.reshape(-1, 1)).toarray()
ohe_region = OneHotEncoder().fit_transform(data['Region'].values.reshape(-1, 1)).toarray()
ohe_agebrackets = OneHotEncoder().fit_transform(data['Age Brackets'].values.reshape(-1, 1)).toarray()

In [9]:
X = data.drop(columns=['ID', 'Education', 'Occupation', 'Commute Distance', 'Region', 'Age Brackets']).values
X = np.column_stack([X, ohe_education, ohe_occ, ohe_commute, ohe_region, ohe_agebrackets])
y = data['Purchased Bike'].values

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential

In [13]:
batch_size = 20

model = Sequential(
    [
        Input((xtrain.shape[1], )),
        Dense(64, activation='tanh'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,545 (17.75 KB)

 Trainable params: 4,545 (17.75 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.compile(loss=BinaryCrossentropy, optimizer='adam', metrics=['accuracy'])

In [16]:
model.fit(xtrain, ytrain, epochs=100)

Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5004 - loss: 0.7124
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4652 - loss: 0.6966 
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5213 - loss: 0.6928 
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5476 - loss: 0.6932 
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5348 - loss: 0.7020 
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4657 - loss: 0.6991 
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4658 - loss: 0.6948 
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5085 - loss: 0.6954 
Epoch 9/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5145 - loss: 0.6945 
Epoch 10/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5188 - loss: 0.6933 
Epoch 11/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5400 - loss: 0.6940 
Epoch 12/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step -

In [17]:
model.evaluate(xtest, ytest)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5417 - loss: 0.6915  


[0.6920275092124939, 0.5299999713897705]